# Estimating the infectious window period from operational data

The simple algorithm in this notebook is based on the idea that you can estimate the duration of the infectious window period (or window of residual risk) from lookback investigation data.

Lookback investigations are triggered when repeat donors seroconvert and trigger an investigation into transfused material collected at the donor's previous donation. The prior donation could have been collected while the donor was already infected (and infectious) but not yet detectable.

Imagine the following situation: 

* 100 donors seroconvert and become HIV-positive
* all 100 had prior donations exactly 105 days before the donation at which HIV was detected
* 1 recipient of material from prior donations became infected and it is confirmed that transfusion transmission occurred
* the least sensitive positive test at the seroconversion donation has a diagnostic delay of 10 days
* the most sensitive negative test at the prior donation has a diagnostic delay of 5 days

Each of these IDIs of 105 days should be adjusted by the diagnostic delays relevant to the time of last negative and first positive donations to derive the interval during which infection could have occurred. In this case to 105+5-10 = 100 days.

In this simple example, the infectious window period associated with the screening strategy in use would be 1% of the duration of the shared infection interval or 1 day. The 1% comes from the 1/100 donors who transmitted the infection. This can be generalized to a situation where each donor has a different IDI by considering the number of transmissions as a Poisson process and treating the adjusted IDIs as "inverse exposure time".

$$IWP = \frac{n}{\sum_{i = 1}^{N} \frac{1}{IDI_{i}} }$$

with $n$ the number of transmissions, $N$ the number of seroconverting donors and $IDI_i$ the i<sup>th</sup> donor's adjusted interdonation interval.

The properties of the Poisson and Chi-square distributions then allow us to obtain confidence intervals on the IWP estimate:

$$IWP_{lb} = \frac{ \left. \chi_{2n,\alpha/2}^{2} \middle/ 2 \right. }{ \sum_{i = 1}^{N} \frac{1}{IDI_{i}} }$$

$$IWP_{ub} = \frac{ \left. \chi_{2n,1-\alpha/2}^{2} \middle/ 2 \right. }{ \sum_{i = 1}^{N} \frac{1}{IDI_{i}} }$$

In [1]:
import numpy as np
import scipy.stats as stats

In [2]:
def residual_risk_operational(n_transmissions,
                              intervals, 
                              negative_diagnositc_day,
                              positive_diagnostic_delay,
                              alpha = 0.05
                             ):
    inverse_intervals = [1/(x + negative_diagnositc_day - positive_diagnostic_delay) for x in intervals]
    total_exposure = sum(inverse_intervals)
    iwp = n_transmissions / total_exposure
    if n_transmissions > 0:
        iwp_lb = stats.chi2.ppf(alpha/2, df=2*n_transmissions)/2/total_exposure
    else:
        iwp_lb = 0.0
    iwp_ub = stats.chi2.ppf(1.0-alpha/2, df=2*(n_transmissions+1))/2/total_exposure
    return (iwp, (iwp_lb, iwp_ub))

## Example with equal IDIs

In [3]:
IDIs = np.repeat(105,100)
residual_risk_operational(1, IDIs, 5, 10)

(0.9999999999999993, (0.02531780798428986, 5.5716433909388945))

## Example with more realistic IDIs

In [4]:
def truncated_normal(mu=0, sigma=1, lower=-3, upper=3):
    return stats.truncnorm(a = (lower - mu)/sigma,
                           b = (upper - mu)/sigma,
                           loc = mu,
                           scale = sigma
                          )

In [5]:
np.random.seed(123)
X = truncated_normal(mu = 0, sigma = 600, lower = 56, upper = 2000)
IDIs = X.rvs(500)

In [6]:
residual_risk_operational(5, IDIs, 5, 10)

(2.733677254711141, (0.887617563599965, 6.379490801132148))